# Brain Splitter
This project is made by the Syntax Error 404 team in order to recognise and classify the MRI scans of patients who are diagnosed with the Alzheimer's disease in order to pinpoint and understand how affected they are by the disease.

## Installation

If these packages/libraries are not already installed in your system, only then shall you run this code block. Without these libraries, the project will not be able to run.

In [ ]:
!pip install tensorflow as tf
!pip install numpy as np
!pip install matplotlib.pyplot as plt

## Importing the modules
We import the required modules and libraries that the project uses

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt

## The Model

In the following code block, we initialise the model by adding the desired convolutional and pooling layers in order to condense the image down and highlight the most important features for the AI.

Each **convolutional** layer reduces the high-dimensionality of images without losing out on data, basically condensing an image down to its most important parts. 
Each **pooling** layer physically reduces the file size and dimensions of the image. When both of these layers work together, the image size is physically halved while the compressed image still retains its original complexity

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    # There are four output neuron. This is for the four classes that our data comes from
    tf.keras.layers.Dense(4, activation='softmax')
])

## Training the model
We use the `ImageDataGenerator` in order to ensure that all images received are internally categorised into training and test data. Furthermore, any new images entered are condensed into the specified file size and hence have a standard aspect ratio.

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer='adam',
              metrics=['accuracy'],
               loss='categorical_crossentropy')

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1/255
train_datagen = ImageDataGenerator(rescale=1/255, validation_split = 0.3)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        './data/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=128,
        # Since we use categorical crossentropy, we have multiple labels
        class_mode='categorical',
        subset='training')

# Flow validation images in batches of 128
validation_generator = train_datagen.flow_from_directory(
        './data/',  # This is the source directory for validation images (gets split in 0.3 ratio with training data)
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=128,
        # Since we use categorical crossentropy, we have multiple labels
        class_mode='categorical',
        subset='validation')

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch= train_generator.samples // 128,
      validation_data = validation_generator,
      validation_steps = validation_generator.samples // 128,
      epochs=12,
      verbose=1)

## Training the Model (Shortcut)

In order to skip the training of the model, you can load it from memory by running the following code block. You can ignore the 2 above code blocks **if you have the model saved** (saves a lot of time). Don't run this block if you have already trained and optimized the model in this session.

In [ ]:
# model = tf.keras.models.load_model('./my_model/my_model')

## Plotting based on training
The following two code blocks focus on generating plots to map out the accuracy and loss on the training set as well as the validation dataset. This shows the relationship between both quantities across both sets of data.

In [ ]:
epochs = range(1, len(history.history['accuracy'])+1)
plt.plot(epochs, history.history['accuracy'], '-o')
plt.plot(epochs, history.history['val_accuracy'], '-o')
plt.title('Validation accuracy vs training accuracy per epoch')
plt.show()

In [ ]:
epochs = range(1, len(history.history['accuracy'])+1)
plt.plot(epochs, history.history['loss'], '-o')
plt.plot(epochs, history.history['val_loss'], '-o')
plt.title('Validation loss vs training loss per epoch')
plt.show()

## Prediction and Classification

The below code block runs the model on our set of test data. It categorises the MRI scans of the brains into 1 of 4 categories which are:
- Non-demented
- Very mildly-demented
- Mildly-demented
- Moderately-demented

Once classified and categorised, the images used for the testing will be *moved into the respective folders* in our directory to remove even this slight trouble involving human labour.

In [ ]:
for filename in os.listdir("./data/test_data"):
    if '.jpg' in filename or '.png' in filename or '.jpeg' in filename:
        path = "./data/test_data/" + filename
        img = tf.keras.utils.load_img(path, target_size=(300, 300))
        x = tf.keras.utils.img_to_array(img)
        x /= 255
        x = np.expand_dims(x, axis=0)
        images = np.vstack([x])
        classes = model.predict(images, batch_size=10)
        if classes[0].tolist().index(max(classes[0])) == 0:
            print("mild :" + str(max(classes[0])*100))
            os.rename(path, "./data/test_data/mild/"+filename)
        elif classes[0].tolist().index(max(classes[0])) == 1:
            print("moderate :" + str(max(classes[0])*100))
            os.rename(path, "./data/test_data/moderate/"+filename)
        elif classes[0].tolist().index(max(classes[0])) == 2:
            print("non :" + str(max(classes[0])*100))
            os.rename(path, "./data/test_data/no/"+filename)
        else:
            print("very mild :" + str(max(classes[0])*100))
            os.rename(path, "./data/test_data/verymild/"+filename)

## Resetting the Test Program

The following block of code resets the test data images to their original uncategorized directories. This should be run before starting every program (but after importing the dependencies as it relies on os) 

In [ ]:
for filename in os.listdir("./data/test_data/mild"):
    os.rename("./data/test_data/mild/"+filename, "./data/test_data/"+filename)
for filename in os.listdir("./data/test_data/moderate"):
    os.rename("./data/test_data/moderate/"+filename, "./data/test_data/"+filename)
for filename in os.listdir("./data/test_data/no"):
    os.rename("./data/test_data/no/"+filename, "./data/test_data/"+filename)
for filename in os.listdir("./data/test_data/verymild"):
    os.rename("./data/test_data/verymild/"+filename, "./data/test_data/"+filename)

## Clean up
This code block is the final one in the program and is run to stop the current session and free up memory resources if required. Do this at the very end of the project to avoid having to train the model again.

In [ ]:
import signal
os.kill(os.getpid(), signal.SIGKILL)